# Домашнее задание

Установить Anaconda и Git. Создать репозиторий на GitHub.

## Easy (+0.1)

Написать функцию, которая на вход принимает список, а возвращает словарь со среднием, медианой и модой, например вот такой: `{"mean": ..., "median": ..., "mode": ...}`.

Пример:

```python

func([0, 1, 1, 10, 5, 4, 3])

# Должно вернуть: {"mean": 3.4285, "median": 3, "mode": 1}

```

In [34]:
#Solution:

def func( arr : list ) -> dict:
    n = len( arr )
    
    sorted_arr = sorted( arr )
    
    mode_value = 0
    mode_count = -1

    i = 0
    j = 0
    while i < n:
        while j < n and sorted_arr[ i ] == sorted_arr[ j ]:
            j += 1
        if j - i > mode_count:
            mode_count = j - i
            mode_value = sorted_arr[ i ]
        i = j

    return {
        "mean": sum( arr ) / n,
        "median": sorted_arr[ n // 2 ],
        "mode": max( arr, key=arr.count )
        }

def test_func():
    assert func( [ 0, 1, 1, 10, 5, 4, 3 ]) == { "mean": 3.4285714285714284, "median": 3, "mode": 1 }
    assert func( [ 0, 1, 2 ] ) == { "mean": 1, "median": 1, "mode": 0 }
    assert func( [ 0, 1, 2, 2 ] ) == { "mean": 1.25, "median": 2, "mode": 2}

test_func()

## Medium (+0.4)

https://www.kaggle.com/datasets/databanditofficial/dota-hero-stats

Написать функцию, которая будет парсить csv-файл dota_hero_stats.csv

Сигнатура функции:

```
def parse_csv(file_content: str) -> List[Dict]
```

Найти персонажа с максимальным количеством ног (сторонние библиотеки использовать нельзя)

In [1]:
#Solution:

def separate_csv_line( line: str ) -> list[ str ]:
    is_in_quotes = False
    result = []
    current = ''
    for c in line:
        if c == '"':
            is_in_quotes = not is_in_quotes
        elif c == ',' and not is_in_quotes:
            result.append( current )
            current = ''
        else:
            current += c
    result.append( current )
    return result

def parse_csv( file_content: str ) -> list[ dict ]:
    lines = file_content.split( '\n' )
    keys = separate_csv_line( lines[ 0 ] )
    table = []

    for line in lines[ 1: ]:
        line_data = separate_csv_line( line )

        if len( line_data ) == len( keys ):
            table.append( dict( zip( keys, line_data ) ) )
    
    return table

def load_heroes_data():
    with open( 'dota_hero_stats.csv' ) as f:
        data = parse_csv( f.read() )
    return data

def find_by_max_legs( data: list[ dict ] ) -> dict:
    return max( data, key = lambda x: int( x.get( 'legs', 0 ) ) )


In [6]:
def test_parse_csv():
    data = load_heroes_data()
    hero = find_by_max_legs( data )
    print( f'Hero with max legs: { hero.get( "localized_name" ) } (have { hero.get( "legs" ) } legs)' )

test_parse_csv()

Hero with max legs: Broodmother (have 8 legs)


## Hard (+ в карму)

Придумать меру близости между двумя персонажами, описать текстом, реализовать кодом. Мера должна использовать все содержательные колонки: attack_type, legs, primary_attr, roles

Найти двух персонажей, которые наиболее близки друг к другу.

Также нельзя использовать сторонние библиотеки.

Все характеристики разделим на три класса:
- первостепенные (имеют наибольший вес): `primary_attr, roles` (в большнстве случаев роль + класс определяют основные характеристики атаки, здоровья и способностей персонажа, поэтому эти классификаторы наиболее ценны)
- второстепенные (средний): `attack_type`
- третистепенные (почти не имеют веса): `legs` (чисто визуальный элемент, который почти не должен влиять на близость)

Веса отдаления распределим так:
- первостепенные - $10$
- второстепенные - $3$ (начинают влиять при сильных откланениях первостепенных)
- третистепенные - $0.1$ (почти не влияют и нужны только в случае полного несовпадения первых двух)

Сравнение по `roles` будем проверять как относительное онколение различных ролей от общего числа ролей $\frac{count(legs_1 \bigtriangleup legs_2)}{count(roles_1) + count(roles_2)}$

Сравнение по `primary_attr` будет булевым: ```primary_attr_1 == primary_attr_2```

Сравнение по `attack_type` будет булевым: ```attack_type_1 == attack_type_2```

Сравнение по `legs` будем проверять как относительное онколение от наибольшего: $\frac{|legs_1 - legs_2|}{\max(legs_1, legs_2)}$

Итоговой мерой отдалйнности будем считать длинну вектора отдалённостей классификаторов: $diff = \sqrt{(\Delta{roles} \cdot primary\_ weight)^2 + (\Delta{primary\_ attr} \cdot primary\_ weight)^2  + (\Delta{attack\_ type} \cdot secondary\_ weight)^2 + (\Delta{legs} \cdot thirdly\_ weight)^2}$

Мерой близости возмём $sim = (1 + diff)^{-1}$

In [14]:
#Solution:

def compare_heroes( hero_1: dict, hero_2: dict ) -> float:
    weights = {
        'roles': 10,
        'primary_attr': 10,
        'attack_type': 3,
        'legs': 0.1
    }
    diffs = {
        'roles': 1,
        'primary_attr': 1,
        'attack_type': 1,
        'legs': 1
    }

    if 'roles' in hero_1 and 'roles' in hero_2:
        diffs[ 'roles' ] = len( set( hero_1[ 'roles' ] ) ^ set( hero_2[ 'roles' ] ) ) / ( len( hero_1[ 'roles' ] ) + len( hero_2[ 'roles' ] ) )

    if 'primary_attr' in hero_1 and 'primary_attr' in hero_2:
        diffs[ 'primary_attr' ] = int( hero_1[ 'primary_attr' ] != hero_2[ 'primary_attr' ] )

    if 'attack_type' in hero_1 and 'attack_type' in hero_2:
        diffs[ 'attack_type' ] = int( hero_1[ 'attack_type' ] != hero_2[ 'attack_type' ] )

    if 'legs' in hero_1 and 'legs' in hero_2:
        hero_1_legs = int( hero_1[ 'legs' ] )
        hero_2_legs = int( hero_2[ 'legs' ] )

        legs_diff = abs( hero_1_legs - hero_2_legs )

        if legs_diff == 0:
            diffs[ 'legs' ] = 0
        else:
            diffs[ 'legs' ] = abs( hero_1_legs - hero_2_legs ) / max( hero_1_legs , hero_2_legs )

    result = 0
    for key in weights:
        result += (weights[ key ] * diffs[ key ]) ** 2

    return (1 + result) ** -0.5

def find_similar_heroes( heroes: list[ dict ] ) -> list[ dict ]:
    result = []
    similarity = 0
    
    for i in range( len( heroes ) ):
        for j in range( i + 1, len( heroes ) ):
            current_similarity = compare_heroes( heroes[ i ], heroes[ j ] )

            if current_similarity > similarity:
                similarity = current_similarity
                result = [ heroes[ i ], heroes[ j ] ]
    
    return result

def find_unique_heroes( heroes: list[ dict ] ) -> list[ dict ]:
    result = []
    similarity = 1
    
    for i in range( len( heroes ) ):
        for j in range( i + 1, len( heroes ) ):
            current_similarity = compare_heroes( heroes[ i ], heroes[ j ] )

            if current_similarity < similarity:
                similarity = current_similarity
                result = [ heroes[ i ], heroes[ j ] ]
    
    return result

In [15]:
def test_find_similar_heroes():
    data = load_heroes_data()
    heroes = find_similar_heroes( data )
    print( f'Most similar heroes: { heroes[ 0 ].get( "localized_name" ) } and { heroes[ 1 ].get( "localized_name" ) } (similarity: { compare_heroes( heroes[ 0 ], heroes[ 1 ] ) })' )

test_find_similar_heroes()

def test_find_unique_heroes():
    data = load_heroes_data()
    heroes = find_unique_heroes( data )
    print( f'Most unique heroes: { heroes[ 0 ].get( "localized_name" ) } and { heroes[ 1 ].get( "localized_name" ) } (similarity: { compare_heroes( heroes[ 0 ], heroes[ 1 ] ) })' )

test_find_unique_heroes()

Most similar heroes: Crystal Maiden and Keeper of the Light (similarity: 1.0)
Most unique heroes: Zeus and Phantom Assassin (similarity: 0.08826178722562064)


## Байка

В реально жизни, конечно, уже написаны парсеры для основных форматов данных, но кто вам гарантирует, что ваши данные будут в нужном формате? Нужно уметь парсить разные странные форматы данных.

Когда-то я работал в компании, которая не умела в DWH и хранила все данные в продовых базах. Ну, "базах". Это были терабайты xml-файлов, поверх которых была настроена репликация и прод умел быстро по ним искать. Конечно же, эти xml-файлы были неструктурированы, пришлось освоить специальный язык для индексации по xml-файлам - xpath - чтобы привести данные в хоть сколько-то табличный вид. Парсер имел отвратительный код, а данные на выходе были еще хуже. А через пару месяцев я уволился оттуда и дальше проект не полетел (надеюсь, это независимые события). Какой вывод? Держите данные в DWH в понятном и удобном формате и не храните данные в проде в xml.